In [1]:
#./code/scripts/my_hdfs.sh fetchdt --renewer null /mnt/data/ON-DEMAND-SPARK/hdfsdt.token
%env PYSPARK_PYTHON /usr/bin/python3
%env PYSPARK_DRIVER_PYTHON /usr/bin/python3
#%env SPARK_DIST_CLASSPATH  /etc/hadoop/conf:/opt/hadoop/share/hadoop/common/lib/*:/opt/hadoop/share/hadoop/common/*:/usr/lib/hadoop-lzo/lib/hadoop-lzo-0.4.19.jar:/usr/lib/hadoop-lzo/lib/hadoop-lzo.jar:/usr/lib/hadoop-lzo/lib/native:/usr/share/aws/aws-java-sdk/aws-java-sdk-bundle-1.11.977.jar:/usr/share/aws/aws-java-sdk/LICENSE.txt:/usr/share/aws/aws-java-sdk/NOTICE.txt:/usr/share/aws/aws-java-sdk/README.md:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/annotations-16.0.2.jar:/usr/share/aws/emr/emrfs/lib/aopalliance-1.0.jar:/usr/share/aws/emr/emrfs/lib/aws-glue-sdk-1.12.0.jar:/usr/share/aws/emr/emrfs/lib/bcprov-ext-jdk15on-1.66.jar:/usr/share/aws/emr/emrfs/lib/emrfs-hadoop-assembly-2.46.0.jar:/usr/share/aws/emr/emrfs/lib/fluent-hc-4.5.6.jar:/usr/share/aws/emr/emrfs/lib/ion-java-1.0.2.jar:/usr/share/aws/emr/emrfs/lib/javax.inject-1.jar:/usr/share/aws/emr/emrfs/lib/jcl-over-slf4j-1.7.21.jar:/usr/share/aws/emr/emrfs/lib/jmespath-java-1.11.852.jar:/usr/share/aws/emr/emrfs/lib/lombok-1.18.4.jar:/usr/share/aws/emr/emrfs/lib/mockito-core-1.10.19.jar:/usr/share/aws/emr/emrfs/lib/objenesis-2.1.jar:/usr/share/aws/emr/emrfs/lib/secret-agent-interface-1.4.0.jar:/usr/share/aws/emr/emrfs/lib/slf4j-api-1.7.21.jar:/usr/share/aws/emr/emrfs/auxlib/*:/usr/share/aws/emr/ddb/lib/emr-ddb-hadoop.jar:/usr/share/aws/emr/goodies/lib/emr-hadoop-goodies.jar:/usr/share/aws/emr/kinesis/lib/emr-kinesis-hadoop.jar:/usr/share/aws/emr/cloudwatch-sink/lib/cloudwatch-sink-2.1.0.jar:/usr/share/aws/emr/cloudwatch-sink/lib/cloudwatch-sink.jar:/opt/hadoop/share/hadoop/tools/lib/*:/usr/lib/hadoop-lzo/lib/*
#%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64
hdfs_endpoint='hdfs://10.0.123.114:8020'
%env HADOOP_TOKEN_FILE_LOCATION /mnt/data/ON-DEMAND-SPARK/hdfsdt.token

env: PYSPARK_PYTHON=/usr/bin/python3
env: PYSPARK_DRIVER_PYTHON=/usr/bin/python3


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType
import random

In [3]:
sparkSession = SparkSession.builder.appName("Calculate Pi using OnDemand Spark") \
.config("spark.dynamicAllocation.enabled", "false") \
.config("fs.default.name", hdfs_endpoint) \
.config("hadoop.security.authentication", "token")\
.config("spark.driver.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \
.config("spark.executorEnv.HADOOP_TOKEN_FILE_LOCATION","/mnt/data/ON-DEMAND-SPARK/hdfsdt.token") \
.config("spark.driverEnv.HADOOP_TOKEN_FILE_LOCATION","/mnt/data/ON-DEMAND-SPARK/hdfsdt.token") \
.getOrCreate()
sc=sparkSession.sparkContext
#.config("spark.executor.extraClassPath", "/opt/hadoop/etc/hadoop:/usr/lib/hadoop-lzo/lib/*:/opt/hadoop/share/hadoop/mapreduce/*:/opt/hadoop/share/hadoop/hdfs/*") \


In [5]:
def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1


In [6]:


columns = StructType([ StructField("name", StringType(), True),
                      StructField("value", DoubleType(), True)
                    ])

count = sc.parallelize(range(0, 1000),1) \
             .filter(inside).count()
data = [("Pi",4.0 * count/1000)]

df = sparkSession.createDataFrame(data=data, schema=columns)

df.show()

+----+-----+
|name|value|
+----+-----+
|  Pi| 3.16|
+----+-----+



In [7]:
#Let us write to a dataset
ds_path = '/user/dominospark/my-pi2'
df.write.csv(ds_path)
#Read it back
sparkSession.read.csv(ds_path).show()


Py4JJavaError: An error occurred while calling o82.csv.
: org.apache.hadoop.security.AccessControlException: org.apache.hadoop.security.AccessControlException: SIMPLE authentication is not enabled.  Available:[TOKEN, KERBEROS]
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.ipc.RemoteException.instantiateException(RemoteException.java:121)
	at org.apache.hadoop.ipc.RemoteException.unwrapRemoteException(RemoteException.java:88)
	at org.apache.hadoop.hdfs.DFSClient.getFileInfo(DFSClient.java:1668)
	at org.apache.hadoop.hdfs.DistributedFileSystem$29.doCall(DistributedFileSystem.java:1576)
	at org.apache.hadoop.hdfs.DistributedFileSystem$29.doCall(DistributedFileSystem.java:157

In [8]:
sc.stop()

In [9]:
!/mnt/code/scripts/my_hdfs.sh dfs -ls

Found 3 items
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:14 .sparkStaging
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 20:45 example
drwxr-xr-x   - dominospark dominospark          0 2021-08-13 21:07 mypi
